# Popular Data Science Questions

Our goal for this project is to determine what content a data science education company should create based on data from [Data Science Stack Exchange](https://datascience.stackexchange.com/).

Instead of scraping the website for data, Stack Exchange provides a public data base that we can use [here](https://data.stackexchange.com/datascience/query/new).